<a href="https://colab.research.google.com/github/yutawatabe/pyCGE/blob/main/EK_Exacthatalgebra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 反実仮想の計算

In [ ]:
# numpy, mathをインポート
import numpy as np # 行列計算のライブラリ
from math import gamma # 価格指数のため

#パラメーターを変更して解きなおすアプローチ

パラメーターをシミュレートして、Eaton Kortumモデルの均衡を計算します。均衡を計算するために賃金を更新する関数を再び導入し、最終的に均衡の賃金、貿易額を求めるところまで関数化します。

In [ ]:
def updatewage(w,theta,sigma,N,L_S,T,ｄ,psi):
  """
    パラメータと経済変数に基づいて賃金を更新します。

    この関数は、入力パラメータと経済変数に基づいて新しい賃金値を計算します。

  パラメータ:
      w (np.ndarray): 異なる国の現在の賃金率の配列。
      theta (float): 貿易の弾力性。
      sigma (float): 財の代替の弾力性。
      N (int): 国の数。
      L_S (np.ndarray): 各国の労働供給。
      T (np.ndarray): 各国の技術水準。
      d (np.ndarray): 国々間の貿易コスト。
      psi (float): 賃金調整パラメータ。

  戻り値:
      w_new (np.ndarray): 更新された国の賃金。
      Z (np.ndarray): 各国の超過労働需要
      P (np.ndarray): 価格指数。
      X (np.ndarray): 国々間の貿易フロー。
  """
  Xn = w * L

  ## 輸入シェアの計算
  pi = np.zeros((N,N)) # 輸入シェア（n国の総消費のうち、i国の財が何割を占めるか）。ここでは輸出国が最初の次元、輸出国が次の次元
  pi_num = np.zeros((N,N)) # 輸入シェアの分子部分
  Phi = np.zeros((N)) # 輸入シェアの分母部分
  for OR,DE in np.ndindex((N,N)):
    pi_num[OR,DE] = T[OR] * (w[OR] * ｄ[OR,DE]) ** (-theta)
    Phi[DE] += pi_num[OR,DE] # Phi[DE] = pi_den[DE] + pi_num[OR,DE]
  for OR,DE in np.ndindex((N,N)):
    pi[OR,DE] = pi_num[OR,DE] / Phi[DE]

  # 価格指数の計算
  P = gamma((theta + sigma - 1) / theta) **(1/(1-sigma)) * Phi ** (-1/theta)

  # 輸入額の計算
  X = np.zeros((N,N))
  for OR,DE in np.ndindex((N,N)):
    X[OR,DE] = pi[OR,DE] * Xn[DE]

  ## 労働超過需要の計算
  L_S = L
  L_D = np.zeros((N))
  for OR,DE in np.ndindex((N,N)):
    L_D[OR] += X[OR,DE] / w[OR] # L_D[OR] = L_D[OR] + X[OR,DE] / w[OR]
  Z = L_D - L_S

  # 賃金更新と基準化
  w_new = w * (1 + psi * (Z / L) )
  wgdp = np.sum(w_new * L)
  w_new = w_new / wgdp

  return w_new,Z,X,P

def solve_eqm(theta,sigma,N,L,T,d,psi,tol,maxiter):

  # 賃金と労働超過需要の初期化
  Z = np.ones((N))
  w = np.ones((N))

  iter = 1
  # 労働超過需要が十分に小さければLoopから抜け出す
  while max(np.abs(Z)) > tol and iter < maxiter:
    iter += 1
    w_old = np.copy(w)
    w,Z,_,_ = updatewage(w,theta=4,sigma=3,N=3,L_S=L,T=T,d=d,psi=0.1)
    if iter % 10 == 0:
      print(iter)
      #print(Z)
      #print(w)

  if iter == maxiter:
    print("Not done")
    return w,Z,P,X
  else:
    w,Z,X,P = updatewage(w,theta=4,sigma=3,N=3,L_S=L,T=T,d=d,psi=0.1)
    return w,Z,P,X


この関数を用いて、均衡をパラメーターから計算しておきます。

In [ ]:
N = 3 # 国の数
theta = 4 # 貿易弾力性のパラメーター
sigma = 3 # 代替の弾力性のパラメーター
T = np.array([1., 1., 1.]) # 技術のパラメーター
L = np.array([1,1.5,1.5]) # 人口
d = np.ones((N,N)) * 1.5 # 貿易費用
for OR in np.ndindex((N)):
  d[OR,OR] = 1 #国内貿易費用は1にしておく。

# 均衡を解く
w,_,P,X = solve_eqm(theta,sigma,N,L,T,d,psi=0.1,tol=0.001,maxiter=1000)
print(w)

10
[0.26058948 0.24647017 0.24647017]


この数値を初期均衡として、反実仮想を計算します。まずパラメーターが既知として、モデルを解きなおしてみる。ここでは貿易自由化を考えます。初期均衡では
$$ \tau_{in} = 1.5 $$
だったところを
$$ \tau_{in}' = 1.2 $$
としておきます（国内貿易費用は1のまま固定）。

In [ ]:
d_new = np.ones((N,N)) * 1.2 # 新しい貿易費用
for OR in np.ndindex((N)):
  d_new[OR,OR] = 1 #国内貿易費用は 1
print(d_new)

dhat = d_new / d
print(dhat)

[[1.  1.2 1.2]
 [1.2 1.  1.2]
 [1.2 1.2 1. ]]
[[1.  0.8 0.8]
 [0.8 1.  0.8]
 [0.8 0.8 1. ]]


モデルを解きなおしてみて、実質賃金（厚生）を比較すると…。



In [ ]:
w_cf,_,P_cf,X_cf = solve_eqm(theta,sigma,N,L,T,d_new,psi=0.1,tol=0.001,maxiter=1000)

U = w / P
U_cf = w / P_cf
print(U_cf/U)

10
[1.09990159 1.08422993 1.08422993]


# Exact Hat Algebraの導入

Exact hat algebraを使って、このTrade liberizationの反実仮想が整合的に求まるかを確認します。具体的には $(w_{sq} / P_{sq}) / (w_{cf} / P_{cf})$という実質賃金がモデルを解きなおした場合と、Exact hat algebraを使った場合で整合的かを確認していきます。

Exact hat algebraのために、総消費と輸入シェアをデータから計算します。

In [ ]:
# 総需要を計算
Xn = np.sum(X,axis=0) # np.sum(X,axis=0)とは、0次元目に沿って（Pythonは0から添え字が始まる）Xを足し上げる、ということ。

# 輸入シェアを計算
pi = np.zeros((N,N))
for OR,DE in np.ndindex((N,N)):
  pi[OR,DE] = X[OR,DE] / Xn[DE]
print(X)
print(Xn)
print(pi)

[[0.17447018 0.04312279 0.04312279]
 [0.04305264 0.27271905 0.05387043]
 [0.04305264 0.05387043 0.27271905]]
[0.26057545 0.36971227 0.36971227]
[[0.66955724 0.11663879 0.11663879]
 [0.16522138 0.73765214 0.14570907]
 [0.16522138 0.14570907 0.73765214]]


とりあえず$\boldsymbol{\hat{w}}$をすべて1と仮置きして、Exact hat algebraの均衡が成立しているかを確認する。

In [ ]:
what = np.ones((N))
print(what)

[1. 1. 1.]


総消費もこれに伴って変化するため、それを計算しておく
$$ X_{i}' = \hat{w}_{i} X_{i} $$

In [ ]:
Xn1 = what * Xn
print(Xn1)

[0.26057545 0.36971227 0.36971227]


この$\boldsymbol{\hat{w}}$から$\boldsymbol{\hat{\pi}}$を計算する
$$ \hat{\pi}_{ni} = \frac{(\hat{w}_i \hat{d}_{ni})^{-\theta}}{ \sum_{k=1}^N \pi_{nk} (\hat{w}_k \hat{d}_{nk})^{-\theta} } $$
そして価格指数も計算する
$$ \hat{P}_n = \left( \sum_{k=1}^N \pi_{nk} (\hat{w}_k \hat{d}_{nk})^{-\theta} \right)^{-1/\theta} $$

In [ ]:
pihat = np.zeros((N,N))
pihat_num = np.zeros((N,N))
pihat_den = np.zeros((N))

for OR,DE in np.ndindex((N,N)):
  pihat_num[OR,DE] = (what[OR] * dhat[OR,DE] )**(-theta)
  pihat_den[DE] += pi[OR,DE] * pihat_num[OR,DE] # pihat_den[DE] = pihat_den[DE] + pihat_num[OR,DE]

for OR,DE in np.ndindex((N,N)):
  pihat[OR,DE] = pihat_num[OR,DE] / pihat_den[DE]

Phat = pihat_den ** (-1/theta)

print(pihat)
print(pihat * pi)
print(np.sum(pihat*pi,axis=0))

[[0.67736806 1.77151002 1.77151002]
 [1.65373062 0.7256105  1.77151002]
 [1.65373062 1.77151002 0.7256105 ]]
[[0.45353669 0.20662679 0.20662679]
 [0.27323165 0.53524814 0.25812507]
 [0.27323165 0.25812507 0.53524814]]
[1. 1. 1.]


ここで新しい貿易額を計算しておく
$$ X'_{ni} = \pi_{ni} \hat{\pi}_{ni} X'_{i} $$

In [ ]:
X1 = np.zeros((N,N))
for OR,DE in np.ndindex((N,N)):
  X1[OR,DE] = pi[OR,DE] * pihat[OR,DE] * Xn1[DE]


仮置きされた$ \boldsymbol{\hat{w}}$の下で、労働市場均衡が成立しているかを確認します。労働市場が均衡しているということは

$$  L^S_{i} = \frac{\sum_{n=1}^N X_{ni}'}{w'_i} $$

となっていなくてはなりません。両辺を$w_i'$で掛け算し、hatで表現すると
$$ \hat{w}_i w_i L_{i} = \sum_{n=1}^N X_{ni}' $$
左を労働供給、右を労働需要ととらえ、この差を超過需要として計算してます

In [ ]:
wL_D = np.zeros((N))
wL_S = w * what * L

for OR,DE in np.ndindex((N,N)):
  wL_D[OR] += X1[OR,DE]

Z = (wL_D - wL_S) / what
print(Z)

[ 0.00804371 -0.00406925 -0.00406925]


モデルを解くのと同様に超過需要から$\boldsymbol{\hat{w}}$をアップデートします。$\psi$を設定しておきます。
$$ \hat{w}_{new,i} = \hat{w}_i * \left(1 + \psi * \frac{Z_i}{X'_{i}}\right) $$

In [ ]:
psi = 0.1 # 収束のスピードをコントロールするパラメーター

what_new = what * (1 + psi * (Z / Xn1) )

ここで新しい$\boldsymbol{\hat{w}}$と古い$\boldsymbol{\hat{w}}$を比較します。ちゃんと超過需要が発生していた国において賃金が高くなっているか確認できるはずです。

In [ ]:
print(Z)
print(what_new - what)

[ 0.01037596 -0.00518798 -0.00518798]
[ 0.00398194 -0.00140325 -0.00140325]


均衡賃金の変化は基準化なしに一意にはきまりません。最後に反実仮想の世界のGDP変化しないように基準化しておきます。

In [ ]:
wgdp1 = np.sum(what_new * Xn)
wgdp = np.sum(Xn)
print(wgdp1)
what_new = what_new / wgdp1 * wgdp
print(what_new)

0.9999999999999998
[1.00398194 0.99859675 0.99859675]


# Exact Hat Algebraを関数化

いままでの流れをまとめると


1.   パラメーターを設定する
2.   仮定した賃金変化をもとに、輸入シェアの変化、貿易額の変化を計算する
3.   新しい貿易額を用いて、労働市場の均衡をチェックする
4.   労働の超過需要を用いて、仮置きした賃金変化を更新していく。

この2から4を**関数**にまとめます。

In [ ]:
def updatewagehat(what,dhat,theta,N,X,psi):

  # 総需要を計算する
  Xn = np.sum(X,axis=0)
  Xn1 = what * Xn

  # 輸入シェアを計算する
  for OR,DE in np.ndindex((N,N)):
    pi[OR,DE] = X[OR,DE] / Xn[DE]

  # 輸入シェアの変化を計算する
  pihat = np.zeros((N,N))
  pihat_num = np.zeros((N,N))
  pihat_den = np.zeros((N))
  for OR,DE in np.ndindex((N,N)):
    pihat_num[OR,DE] = (what[OR] * dhat[OR,DE] )**(-theta)
    pihat_den[DE] += pi[OR,DE] * pihat_num[OR,DE]
  for OR,DE in np.ndindex((N,N)):
    pihat[OR,DE] = pihat_num[OR,DE] / pihat_den[DE]

  # 価格指数の変化を計算する
  Phat = pihat_den ** (-1/theta)

  # 新しい貿易額を計算する
  X1 = np.zeros((N,N))
  for OR,DE in np.ndindex((N,N)):
    X1[OR,DE] = pi[OR,DE] * pihat[OR,DE] * Xn1[DE]

  # 新しい均衡での労働市場均衡を計算する
  wL_D = np.zeros((N))
  wL_S = w * what * L
  for OR,DE in np.ndindex((N,N)):
    wL_D[OR] += X1[OR,DE]
  Z = (wL_D - wL_S) / what

  # 賃金を更新、基準化する
  what_new = what * (1 + psi * (Z / Xn1) )
  wgdp1 = np.sum(Xn1)
  wgdp = np.sum(Xn)
  what_new = what_new / wgdp1 * wgdp

  return what_new,Z,Phat,X1

ここでちゃんと結果が前の関数なしで書いたiterationと一致するかを確認します。

In [ ]:
what_newfunc,_,_,_ = updatewagehat(what,dhat=dhat,theta=theta,N=N,X=X,psi=psi)
print(what_newfunc)
print(what_new)

[1.00869963 0.99693413 0.99693413]
[1.00398194 0.99859675 0.99859675]


最後にwhile-loopを用いて$\boldsymbol{\hat{w}}$が均衡条件を満たすまで更新していきます

In [ ]:
tol = 0.0001
iter = 1

Z = np.ones((N))
what = np.ones((N))
while max(np.abs(Z)) > tol and iter < 100:
  iter += 1
  what_old = np.copy(what)
  what,Z,_,_ = updatewagehat(what,dhat=dhat,theta=theta,N=N,X=X,psi=0.1)
  if iter % 10 == 0:
    print(iter)
    print(Z)
    print(what)

  if max(np.abs(Z)) < tol:
    print("Done!")
    print(iter)
    print(Z)
    print(what)


10
[ 8.34014292e-05 -4.22723491e-05 -4.22723491e-05]
[1.00868191 0.99694031 0.99694031]
Done!
10
[ 8.34014292e-05 -4.22723491e-05 -4.22723491e-05]
[1.00868191 0.99694031 0.99694031]


これが均衡を計算しなおした値と一致するか確認します

$$ \hat{U}_i = \hat{w}_i / \hat{P}_i $$

In [ ]:
Uhat = what / Phat
Uhat_rs = U_cf/U
what_rs = w_cf/w

print(Uhat)
print(Uhat_rs)
print(what)
print(what_rs)

[1.10943559 1.08091798 1.08091798]
[1.09990159 1.08422993 1.08422993]
[1.00868191 0.99694031 0.99694031]
[1.00861942 0.99696227 0.99696227]
